In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
# input_file_path = "../singleTest/single_rawData.h5"
input_file_path = "../positive/rawData/positive_rawData_520.h5"
input_fov, input_height, input_width = read_file_parameter(input_file_path) 

image_width = 200
image_height = 200
fov = 35
# Initialize detector
mydetector = detector(0.01, fov, image_width, image_height)

# Read and process photons
photons, failed_lines = read_raw_data(input_file_path)

print(len(photons))
for photon in photons:
    mydetector.photon_to_dector(photon)
pixels,image_width,image_height = mydetector.output_to_array()

range_min = 2000
range_max = 3300
input_bin_number = 36
myRange = [range_min,range_max]
image, illegal_photon, stamped_histogram, stamped_collosioin = form_histogram_image(pixels, image_height,image_width,bin_number=input_bin_number,range_distance=myRange)
display_image(image,None,distance_range=myRange)

In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*

input_file_path = "../STM32Lidar/positive/pixelizedData/positive_pixelized_1.h5"

range_min = 800
bin_width = 18 
input_bin_number = 75


range_max = range_min + input_bin_number * bin_width
myRange = [range_min,range_max]
pixels, image_width, image_heigh = decode_file(input_file_path)

image, illegal_photon, stamped_histogram, stamped_collosioin = form_histogram_image(pixels, image_heigh,image_width,bin_number=input_bin_number,range_distance=myRange)

display_image(image,None,distance_range=myRange)